## Imports

In [1]:
import pandas as pd
import numpy as np
import configparser
import os
import sys

In [2]:
cwd = os.getcwd()

In [3]:
module_path = os.path.join(cwd, '..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import financialstatements

## Config

In [5]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\Tony\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Credit Card Functions

In [6]:
def joining_gl_codes(df, coa_mcc_df):
    df['Amount'] = df['Amount'].abs()
    df = df.pivot_table(index = ['Date', 'Name', 'Memo'], columns='Transaction', values='Amount')
    df = df.reset_index()
    df[['Memo', 'MCC', 'Blank']] = df['Memo'].str.split(';', expand=True)
    df['MCC'] = df['MCC'].str[-4:]
    df['MCC'] = df['MCC'].astype('int')
    df = pd.merge(df, 
              coa_mcc_df, 
              how='left',
              on='MCC')
    return df

In [7]:
def processing_df(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Description'] = df['MCC_Description'] + ': ' + df['Name']
    df = df[['Date', 'GL_Code', 'Account', 'Description','DEBIT', 'CREDIT']]
    df = df.round(2)
    df = df.sort_values(by='Date').reset_index(drop=True)
    df['Order_Col'] = df.index + 1
    df['Sub_Order_Col'] = np.where(df['DEBIT'].isnull(), 2, 1)
    return df

In [8]:
def creating_credit_entries(df):
    df = df[df['CREDIT'].isnull()].reset_index(drop=True).copy()
    df['GL_Code'] = 200101
    df['Account'] = 'EdwardJones MasterCard'
    df['Sub_Order_Col'] = 2
    df = df.rename(columns={'DEBIT':'CREDIT', 'CREDIT': 'DEBIT'})
    return df[['Date', 'GL_Code', 'Account', 'Description','DEBIT','CREDIT','Order_Col', 'Sub_Order_Col']]

In [9]:
def creating_debit_entries(df):
    df = df[df['DEBIT'].isnull()].reset_index(drop=True).copy()
    df['GL_Code'] = 200101
    df['Account'] = 'EdwardJones MasterCard'
    df['Sub_Order_Col'] = 1
    df = df.rename(columns={'DEBIT':'CREDIT', 'CREDIT': 'DEBIT'})
    return df[['Date', 'GL_Code', 'Account', 'Description','DEBIT','CREDIT','Order_Col','Sub_Order_Col']]

## Reading Data

### Credit Card Data

In [10]:
# Creating directory to credit card data
CREDITCARD_DIRECTORY = os.path.join(cwd,config.get("data_inputs_directory", "CREDITCARD_DIRECTORY"))

In [11]:
paths = financialstatements.creating_input_paths(CREDITCARD_DIRECTORY)

In [12]:
df = financialstatements.creating_df(paths)

### Table Data

In [13]:
COA_DATA = config.get('table_files', 'COA_DATA')
MONTH_DATA = config.get('table_files', 'MONTH_DATA')

In [14]:
coa_mcc_df = pd.read_excel(COA_DATA, sheet_name='coa_mcc_link_table')

In [15]:
month_df = pd.read_excel(MONTH_DATA)

## ETL

In [16]:
df = joining_gl_codes(df, coa_mcc_df)

In [17]:
# Logic gate for checking nulls in the GL_Code column
if financialstatements.missing_gl_check(df) != 0:
    print("GL_Codes are missing, need to update the coa and mcc link table")
    financialstatements.missing_mcc(df)
else:
    print("No GL_Codes missing")

No GL_Codes missing


In [18]:
df = processing_df(df)

In [19]:
credit_entries_df = creating_credit_entries(df)

In [20]:
debit_entries_df = creating_debit_entries(df)

In [21]:
df = pd.concat([df, credit_entries_df, debit_entries_df]).reset_index(drop=True)

In [22]:
df = df.sort_values(by=['Order_Col','Sub_Order_Col']).reset_index(drop=True)

In [23]:
if financialstatements.debit_credit_check(df) == True:
    print("Debits equal credits. Proceed to next phase")
else:
    print("Something went wrong")

Debits equal credits. Proceed to next phase


In [24]:
type = 'creditcard'

In [25]:
financialstatements.creating_output(df, month_df, type, config, cwd)